In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import os
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.layers import *
from keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

C:\Users\Suryadev Zala\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
folder_path='./train'

In [3]:
os.listdir(folder_path)

['NORMAL', 'PNEUMONIA']

In [4]:
labels=[]
X=[]


for i in os.listdir(folder_path):
    if i=='NORMAL':
        label=0
    else:
        label=1
    file_path=os.path.join(folder_path,i)
    for j in os.listdir(file_path):
        final_path=os.path.join(file_path,j)
        img=cv2.imread(final_path)
        img=cv2.resize(img,(224,224))
        img=np.array(img)
        x=img.reshape(224,224,3)
        X.append(x)
        labels.append(label)
        

In [5]:
labels_test=[]
X_test=[]

folder_path_test='./test'

for i in os.listdir(folder_path_test):
    if i=='NORMAL':
        label=0
    else:
        label=1
    file_path_test=os.path.join(folder_path_test,i)
    for j in os.listdir(file_path_test):
        final_path_test=os.path.join(file_path_test,j)
        img=cv2.imread(final_path_test)
        img=cv2.resize(img,(224,224))
        img=np.array(img)
        x=img.reshape(224,224,3)
        X_test.append(x)
        labels_test.append(label)

In [6]:
X=np.array(X)
labels=np.array(labels)
X_test=np.array(X_test)
labels_test=np.array(labels_test)
X[0].shape

(224, 224, 3)

In [7]:
model=VGG16(weights='imagenet',
           input_shape=(224,224,3),
           include_top=False)

In [8]:
for i in model.layers:
    i.trainable=False

In [9]:
X=preprocess_input(X)
X_test=preprocess_input(X_test)

In [10]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
flat=Flatten()
dense1=Dense(512 ,activation='relu')
dense2=Dense(512 ,activation='relu')
output=Dense(2 ,activation='softmax')

In [12]:
model2=Sequential([model,flat,dense1,dense2,output])

In [13]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 27823938 (106.14 MB)
Trainable params: 13109250 (50.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [14]:
model2.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

In [16]:
model2.fit(X,labels,validation_split=0.2, batch_size=32, callbacks=[es])



131/131 [==============================] - 687s 5s/step - loss: 3.5190 - accuracy: 0.9382 - val_loss: 3.2882 - val_accuracy: 0.8898


In [17]:
img=cv2.imread('./val/PNEUMONIA/person1954_bacteria_4886.jpeg')
img=cv2.resize(img,(224,224))
img=np.array(img)
x=img.reshape(1,224,224,3)

In [18]:
img.shape

(224, 224, 3)

In [19]:
x.shape

(1, 224, 224, 3)

In [20]:
y_pred=model2.predict(x)

1/1 [==============================] - 1s 533ms/step


In [21]:
y_pred.argmax()

1

In [22]:
from joblib import dump
dump(model2,'./../../Brain tumor detection/Django/deploy/SavedModels/Chest_X-Ray-VGG16.joblib')

['./../../Brain tumor detection/Django/deploy/SavedModels/Chest_X-Ray-VGG16.joblib']